In [38]:
import json
from requests import Response, get

url = "https://bgapidatafeed.vps.com.vn/listvn30"

response: Response = get(url, timeout=10)
if response.status_code == 200:
    symbols: list[str] = response.json()
    with open("vn30.json", "w") as vn30:
        json.dump(symbols, vn30, indent=2)

KeyboardInterrupt: 

In [ ]:
import datetime
import math
import os
import pandas as pd
import requests

def get_history(symbol: str):
    now = math.floor(datetime.datetime.now().timestamp())
    url = f"https://histdatafeed.vps.com.vn/tradingview/history?symbol={symbol}&resolution=1D&from=0&to={now}"
    response = requests.get(url)
    history_data: dict = response.json()
    timestamp: list[int] = history_data.get("t", [])
    open_price: list[float] = history_data.get("o", [])
    high_price: list[float] = history_data.get("h", [])
    low_price: list[float] = history_data.get("l", [])
    close_price: list[float] = history_data.get("c", [])
    volumn: list[int] = history_data.get("v", [])
    history: list[dict] = []
    if len(timestamp) > 0:
        for index, timestamp_item in enumerate(timestamp):
            date = datetime.datetime.fromtimestamp(timestamp_item).date().strftime("%Y-%m-%d")
            history.append(
                {
                    "symbol": symbol,
                    "date": date,
                    "timestamp": timestamp_item,
                    "open": open_price[index],
                    "high": high_price[index],
                    "low": low_price[index],
                    "close": close_price[index],
                    "volume": volumn[index],
                }
            )
        history_data_frame = pd.DataFrame(history)
        history_data_frame["date"] = pd.to_datetime(history_data_frame["date"])
        folder_path = "./history"
        file_path = f"{folder_path}/{symbol}.csv"
        os.makedirs(folder_path, exist_ok=True)
        history_data_frame.to_csv(file_path, index=False)

In [ ]:
for symbol in symbols:
    get_history(symbol)

In [43]:
import json

with open("vn30.json", "r") as f:
    vn30 = json.load(f)

data = []

today = pd.Timestamp.today()

# Filter for rows within the last 3 months
three_months_ago = today - pd.DateOffset(months=12)

for symbol in vn30:
    data_frame = pd.read_csv(f"./history/{symbol}.csv")
    data_frame['date'] = pd.to_datetime(data_frame['date'])
    filtered_df = data_frame[data_frame['date'] >= three_months_ago]
    lowest_close_price = filtered_df['close'].min()
    latest_close_price = filtered_df['close'].iloc[-1]
    delta_close_price = latest_close_price - lowest_close_price
    delta_close_price_percent = (delta_close_price / lowest_close_price) * 100 if lowest_close_price else 0
    data.append({
        "symbol": symbol,
        "lowest_close_price": lowest_close_price,
        "latest_close_price": latest_close_price,
        "delta_close_price": delta_close_price,
        "delta_close_price_percent": delta_close_price_percent
    })

df = pd.DataFrame(data)
df = df.sort_values(by=["delta_close_price_percent", "latest_close_price"], ascending=[True, True])
df

,symbol,lowest_close_price,latest_close_price,delta_close_price,delta_close_price_percent
21,TPB,11.900,11.90,0.000,0.000000
28,VPB,16.250,16.25,0.000,0.000000
18,SSI,22.100,22.10,0.000,0.000000
0,ACB,22.150,22.15,0.000,0.000000
9,HPG,22.900,22.90,0.000,0.000000
7,GVR,25.750,25.75,0.000,0.000000
14,PLX,33.250,33.25,0.000,0.000000
2,BID,33.500,33.50,0.000,0.000000
15,SAB,42.600,42.60,0.000,0.000000
12,MSN,54.000,54.00,0.000,0.000000
